# Convolutional Neural Network

### Importing the libraries

In [2]:
import tensorflow as tf
import tensorflow.keras.utils as tf_utils
from tensorflow.keras import layers

In [3]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

In [4]:
# Ensure TensorFlow is using the GPU if available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

### Preprocessing the Training set

In [5]:


#load and transform the training dataset
#augment the images with random horizontal flip and random rotation, zoom and shear

train_dataset = tf_utils.image_dataset_from_directory(
    "../dataset/training_set",
    batch_size=32,
    image_size=(64, 64),
    label_mode="binary"
    
)



# Data augmentation using Keras preprocessing layers
#we do this seperately to avoid applying the same augmentation to validation/test datasets
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])


# Apply data augmentation to the training dataset
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
).map(
    lambda x, y: (tf.cast(x, tf.float32) / 255.0, y), # Normalize the images, always need to do this with images so the pixels are between 0 and 1
    num_parallel_calls=tf.data.AUTOTUNE
)

Found 8000 files belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_dataset = tf_utils.image_dataset_from_directory(
    "../dataset/test_set",
    batch_size=32,
    image_size=(64, 64),
    label_mode="binary"
).map(
    lambda x, y: (tf.cast(x, tf.float32) / 255.0, y), #cast to float32 and normalize
    num_parallel_calls=tf.data.AUTOTUNE
)



Found 2000 files belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
#sequential in models is the neural network class

#initialize a neural network
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
#add convolutional layers to the model
#in layers are the different layers for the neural network
#number of filters, size of filters and activation function with relu
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=4, activation='relu', input_shape=(64, 64, 3))) #resized to 64x64 and we are using 3 channels (RGB)
#when we add a first layer, we need to specify the input shape, after that whenever adding it will be infered

c:\Users\ayb19\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=4, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #units is number of neurons in the layer

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(
    train_dataset,
    epochs=100,#how many times it was trained
    validation_data=test_dataset, 
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - accuracy: 0.5009 - loss: 0.7173 - val_accuracy: 0.5640 - val_loss: 0.6715
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step - accuracy: 0.5688 - loss: 0.6802 - val_accuracy: 0.6170 - val_loss: 0.6580
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 114ms/step - accuracy: 0.6240 - loss: 0.6571 - val_accuracy: 0.6605 - val_loss: 0.6271
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 105ms/step - accuracy: 0.6287 - loss: 0.6393 - val_accuracy: 0.7005 - val_loss: 0.5836
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 104ms/step - accuracy: 0.6676 - loss: 0.6168 - val_accuracy: 0.7035 - val_loss: 0.5790
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.6991 - loss: 0.5859 - val_accuracy: 0.6590 - val_loss: 0.6505
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - accuracy: 0.6946 - loss: 0.5778 - val_accuracy: 0.7335 - val_loss: 0.5430
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 62ms/step - accuracy: 0.7165 - loss

## Part 4 - Making a single prediction

In [38]:
#predict based on a random image from the test_dataset

import random
import pathlib
import numpy as np



path = pathlib.Path("../dataset/test_set") / f"{(rand_label := random.choice(["cats", "dogs"]))}/{rand_label[:-1]}.{random.randint(4000, 5000)}.jpg" 
print(rand_label)
print(path)

random_image = tf.keras.utils.load_img(path, target_size=(64, 64))
random_image = tf.keras.utils.img_to_array(random_image)
random_image = np.expand_dims(random_image, axis=0)  # Add batch dimension
prediction = cnn.predict(random_image)
print(prediction)
print(f"Prediction for {path.name}: {'Dog' if prediction[0][0] > 0.5 else 'Cat'}")






cats
..\dataset\test_set\cats\cat.4305.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[1.]]
Prediction for cat.4305.jpg: Dog
